# <span style="color:#9c8f8f"> 75.06/95.58 Organización de Datos</span>
# <span style="color:#9c8f8f"> Análisis exploratorio: Real or Not? NLP with Disaster Tweets</span>

# <center>FEATURE ENGINEERING</center>
# <center>Random Forest</center>

In [1]:
# Cargo librerias

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split

In [2]:
# Cargo train and test

dtype_train = {"id": np.int32, "keyword": "category", "target" : int}
train = pd.read_csv("original_data/train.csv", dtype = dtype_train, encoding='UTF_8')

In [3]:
# Creo dataframe con target

target_train = train[["id","target"]]

In [4]:
DIR_CSV = "features/"

# I) Random Forest

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. 

# II) Auxiliary Functions

In [5]:
dtype_train = {"id": np.int32, "keyword": "category", "target" : int}
train = pd.read_csv("original_data/train.csv", dtype = dtype_train, encoding='UTF_8')

target_train = train[["id","target"]]
    
def get_clean_values(df):
    
    if("target" in list(df.columns)):
        del df["target"]
    df = df.merge(target_train, how="inner", on="id")   
    
    X = df.copy().fillna(0)
    y = df["target"].copy()
    
    if("target" in list(X.columns)):
        del X['target']
    if("id" in list(X.columns)):
        del X['id']
    if("Unnamed: 0" in list(X.columns)):
        del X["Unnamed: 0"]
    
    return X,y

In [13]:
# m cantidad de features con más relevancia

def random_forest_classifier_files(previous_dir, files, m_value):
    
    # contiene una lista por archivo con feature_importances, m_index, m_feature_importances, m_feature_name
    locations_others_info = []
    
    for i in range(len(files)):
        
        # cargo archivo
        df = pd.read_csv(previous_dir + files[i], low_memory=False)

        # Separo mi data
        X, y = get_clean_values(df)
        
        # Verifico m
        m = m_value if (m_value < len(X.columns)) else len(X.columns)

        # Aplico random forest
        feature_importances, m_index, m_feature_importances, m_feature_name = random_forest_feature_search(X,y,m)
        locations_others_info.append([feature_importances, m_index, m_feature_importances, m_feature_name])

        # Imprimo informacion
        print_feature_search(m_feature_name, m_feature_importances, files[i])
    
    return locations_others_info

In [19]:
def random_forest_feature_search(X,y,m):
    
    # Model creation
    
    rf_model = RandomForestClassifier(random_state = 1, n_estimators = 4, max_depth = 4)
    rf_model.fit(X,y)
    #clf = tree.DecisionTreeRegressor()     # DecisionTreeClassifier
    #clf = clf.fit(X, y)
    
    # Feature importance
    feature_importances = rf_model.feature_importances_
    #feature_importances = clf.feature_importances_

    # get index of m most important features sorted
    m_index = np.argpartition(feature_importances, -m)[-m:]
    m_index = m_index[np.argsort(feature_importances[m_index])][::-1]
    
    # Get the m most important features of locations
    m_feature_importances = feature_importances[m_index]
    
    # Get m feature name
    m_feature_name = X.columns[m_index]
    
    return feature_importances, m_index, m_feature_importances, m_feature_name

In [15]:
def print_feature_search(feature, importance, file):
    
    print("{}\n".format(file.upper()))
    print("{: >20} {: >20}\n".format("feature","importance"))
    for i in range(len(feature)):
        print("{: >20}\t\t{: >20}".format(feature[i],importance[i]))
    print("\n\n")

# III) Random Forest

In [21]:
## aver
m = 30

## 1. Keywords

In [20]:
files_keywords = ["features_keywords_in_text_encoded.csv", "features_keywords_numerical.csv"]
DT_keywords = random_forest_classifier_files(DIR_CSV + 'train/' ,files_keywords, m)

FEATURES_KEYWORDS_IN_TEXT_ENCODED.CSV

             feature           importance

             K_storm		 0.12708425146055213
              K_bomb		 0.12489566686839923
           K_typhoon		 0.12320326417651224
          K_wildfire		 0.11457562077548372
              K_fire		 0.11319673599340546
        K_derailment		 0.08896741193733057
      K_suicide bomb		 0.06257204723958362
          K_outbreak		 0.05774217160102982
        K_earthquake		 0.05446390582797898
   K_suicide bombing		  0.0407023522216952
          K_massacre		 0.03270058757652258
      K_thunderstorm		 0.02241836963135475
              K_ruin		0.009864902344619031
         K_windstorm		0.009549317561139118
              K_army		0.005013454171291252
           K_thunder		0.004765579655295246
         K_emergency		0.002694112533872206
         K_hurricane		0.0015923389292918546
             K_fatal		0.0015314829624869463
              K_hail		 0.00104036357520937
            K_attack		  0.0007979754506291
           K_

## 2. Text

In [22]:
files_text = [
    'features_tagged.csv',
    'features_tagged_smooth.csv',
    'features_hashtags.csv',
    'features_hashtags_smooth.csv',
    'features_text_numerical.csv',
    'features_arrobas_numerical.csv',
    'features_hashtags_numerical.csv'
]

DT_text = random_forest_classifier_files(DIR_CSV + 'train/', files_text, m)

FEATURES_TAGGED.CSV

             feature           importance

     organicallyrude		 0.10305993139811938
            whvholst		 0.08365786627259202
         davidvitter		   0.077014122121972
              usagov		  0.0769736150207783
         mrmikeeaton		 0.07683209155149866
       cityofcalgary		 0.07323273336458684
            alanhahn		  0.0680589288876612
             115film		 0.06051150012040466
              ashayo		 0.06044780380449162
                wews		 0.05770744000200056
      frackfreelancs		0.057661941363351114
       meetkakarotto		 0.05761649651267631
          nickcannon		 0.05438274249400498
         apollobrown		 0.05432559525374696
            ajabrown		  0.0385171918321154
             evacide		                 0.0
             evaaasr		                 0.0
         evahanderek		                 0.0
                euro		                 0.0
        eunice_njoki		                 0.0
            etribune		                 0.0
          ethereal_7		           

## 3. Locations


In [23]:
files_location = [
    'features_location_bow.csv',
    'features_location_tf_idf.csv',
    'features_location_numerical.csv',
    'features_location_in_text_numerical.csv',
    'features_location_in_text_numerical.csv'
]

DT_location = random_forest_classifier_files(DIR_CSV + 'train/' , files_location, 10)

FEATURES_LOCATION_BOW.CSV

             feature           importance

          country_in		  0.3046826127875726
          country_pk		 0.16001937576770225
          country_ph		 0.12659986319268274
          country_ie		 0.10976289880724549
          country_it		 0.07204733210987213
          country_vn		 0.05769578221543165
          country_mx		 0.03762226675253353
          country_iq		 0.03245143615924557
          country_cy		 0.03133127555294549
          country_au		 0.02686729088542711



FEATURES_LOCATION_TF_IDF.CSV

             feature           importance

          country_ch		 0.14203586846016347
          country_jm		 0.12792657114302713
          country_au		 0.08862505194188176
          country_er		 0.08658532543988601
          country_je		 0.08620386369644675
          country_mx		 0.07055298155287756
          country_pk		 0.07051501270590511
          country_kr		 0.06594290842311462
          country_jp		0.049520277288717655
          country_cf		 0.046230315977

In [24]:
files_location_places = ['features_city_encoded.csv',
    'features_city_in_text_encoded.csv',
    'features_country_encoded.csv',
    'features_country_in_text_encoded.csv',
    'features_county_encoded.csv',
    'features_county_in_text_encoded.csv',
    'features_state_encoded.csv',
    'features_state_in_text_encoded.csv',
    'features_continent_encoded.csv',
    'features_continent_in_text_encoded.csv']

DT_location_places = random_forest_classifier_files(DIR_CSV + 'train/' , files_location_places, m)

FEATURES_CITY_ENCODED.CSV

             feature           importance

        city_Karachi		 0.13105868350539346
        city_Calgary		 0.12490306554278768
  city_Santo Domingo		 0.10236218179367049
        city_Concord		 0.09021717938320016
       city_Oklahoma		 0.07364493666320328
     city_Broomfield		 0.06412306537949407
        city_Menasha		 0.06410728471646147
       city_Cornwall		 0.06148193336560863
    city_Baton Rouge		 0.06020164555850757
           city_Lugo		 0.04812471324084117
      city_Milwaukee		0.045248660108174604
         city_Newark		 0.03977484235431424
     city_Burlington		0.038093216577751714
   city_Cedar Rapids		0.029856519489638494
         city_Durham		 0.02680207232095305
   city_Garden Grove		                 0.0
     city_Greensboro		                 0.0
   city_Grand Rapids		                 0.0
    city_Garden City		                 0.0
     city_Fort Wayne		                 0.0
       city_Zaragoza		                 0.0
         city_Frisco		     

FEATURES_STATE_IN_TEXT_ENCODED.CSV

             feature           importance

    state_California		  0.2193647515089941
            state_GA		 0.13000888193386714
            state_AL		 0.10316044913627781
      state_Illinois		 0.09701757568265043
        state_Hawaii		 0.08784960350630378
    state_Washington		  0.0781876571556813
            state_CA		 0.06937715624428935
            state_TN		  0.0624835018638922
state_North Carolina		 0.04695518092104252
            state_KY		0.034821769037955266
            state_TX		 0.03444110420886185
            state_OH		0.020812369654122382
       state_Alabama		0.014023863824278275
            state_VA		0.0014865978324373225
       state_Florida		9.537489346288692e-06
       state_Indiana		                 0.0
            state_KS		                 0.0
         state_Idaho		                 0.0
            state_IN		                 0.0
        state_Kansas		                 0.0
            state_MI		                 0.0
            stat

## 4. Links

In [25]:
files_links = ['features_links_numerical.csv', "features_domain_bow.csv", "features_domain_tf_idf.csv", "features_domain_smooth_tf_idf.csv"]
DT_links = random_forest_classifier_files(DIR_CSV + 'train/' , files_links, m)

FEATURES_LINKS_NUMERICAL.CSV

             feature           importance

          links_cant		  0.9138210203764556
   cant_failed_links		 0.08617897962354433
                id.1		                 0.0



FEATURES_DOMAIN_BOW.CSV

             feature           importance

          domain_bbc		 0.20787018730354986
  domain_feedsportal		  0.1958846732554367
       domain_amazon		 0.09050709680322017
     domain_blogspot		 0.07611919293027597
   domain_microsites		 0.06930397720121599
           domain_it		  0.0637928262118191
       domain_tumblr		 0.05555903005273852
domain_businessinsider		 0.05299540602103566
   domain_gamesradar		 0.03060854678924622
    domain_instagram		0.028688880741873823
     domain_grownnon		 0.01926221796714702
        domain_music		0.016934061501282918
        domain_drops		0.016798330446702717
        domain_hedge		0.016055859237181096
domain_innerlifemining		0.010335824275054493
      domain_goceleb		0.010300295886766132
      domain_weather		0.00921839425

## 5. Tags

In [26]:
files_tags = ["features_tags_bow.csv", "features_tags_tf_idf.csv"]
DT_tags = random_forest_classifier_files(DIR_CSV + 'train/' , files_tags, m)

FEATURES_TAGS_BOW.CSV

             feature           importance

          tag_bomber		 0.17545889435260237
            tag_fire		 0.12039654042923374
      tag_california		 0.11582514209691967
           tag_japan		 0.07207258031930378
      tag_earthquake		  0.0649533820653236
         tag_bombing		 0.06427951506310826
     tag_anniversary		  0.0607951149294216
             tag_lol		 0.05598664595235563
            tag_kill		0.052846151338704135
          tag_atomic		 0.05236062757326765
         tag_calgary		 0.04176095019720612
          tag_family		 0.03307223945041979
       tag_swallowed		0.026393565959973348
         tag_reunion		0.022939097954176826
            tag_ebay		0.015232769993862916
            tag_flag		 0.01036046336609105
           tag_white		0.005532404589212824
             tag_god		0.004618335483133777
         tag_collide		0.0025777940716142284
           tag_photo		0.001585221776620055
     tag_devastation		0.0004512670505094319
              tag_mh		0.00025

# IV) Conclusion

In [27]:
list_of_files = [
    files_keywords,
    files_text,
    files_location,
    files_location_places,
    files_links,
    files_tags,
]

In [28]:
# Information of random forest for m or less features

DT = [
    DT_keywords,
    DT_text,
    DT_location,
    DT_location_places,
    DT_links,
    DT_tags,
]

In [29]:
# Features importance less than min_importance function

def filter_importance(DT, min_importance):
    feature_importances, m_index, m_feature_importances, m_feature_name = DT
    
    res = []
    for i in range(len(m_feature_importances)):
        if(m_feature_importances[i]>= min_importance):
            res.append(m_feature_name[i])
    return res

In [30]:
# Creates a dataframe with all the m features for a csv filtered 

min_importance = 0.00001

df_total = []

for i in range(len(list_of_files)):
    
    for j in range(len(list_of_files[i])):
        
        df = pd.read_csv(DIR_CSV + 'train/'+ list_of_files[i][j] ,header=0)
        
        cols = list(filter_importance(DT[i][j], min_importance))
        
        cols.append("id")
                               
        df_total.append(df[cols])

In [31]:
for df in df_total:
    print(list(df.columns),'\n')

['K_storm', 'K_bomb', 'K_typhoon', 'K_wildfire', 'K_fire', 'K_derailment', 'K_suicide bomb', 'K_outbreak', 'K_earthquake', 'K_suicide bombing', 'K_massacre', 'K_thunderstorm', 'K_ruin', 'K_windstorm', 'K_army', 'K_thunder', 'K_emergency', 'K_hurricane', 'K_fatal', 'K_hail', 'K_attack', 'K_tornado', 'K_terrorist', 'K_evacuate', 'K_displace', 'id'] 

['keywords_mean', 'keywords_quantity', 'keyword_frequency', 'keyword_is_hashtag', 'has_keyword', 'text_contains_keyword', 'id'] 

['organicallyrude', 'whvholst', 'davidvitter', 'usagov', 'mrmikeeaton', 'cityofcalgary', 'alanhahn', '115film', 'ashayo', 'wews', 'frackfreelancs', 'meetkakarotto', 'nickcannon', 'apollobrown', 'ajabrown', 'id'] 

['organicallyrude', 'whvholst', 'davidvitter', 'usagov', 'mrmikeeaton', 'cityofcalgary', 'alanhahn', '115film', 'ashayo', 'wews', 'frackfreelancs', 'meetkakarotto', 'nickcannon', 'apollobrown', 'ajabrown', 'id'] 

['nowplaying', 'earthquake', 'abstorm', 'iran', 'california', 'socialnews', 'bioterrorism',

In [32]:
df_final = train[["id", "target"]]

In [33]:
# Mergeo todos los dataframe

for df in df_total:
    df_final = df_final.merge(df, on="id", how="left")

len(df_final.columns)

395

In [35]:
X,y = get_clean_values(df_final)
DT_final = random_forest_feature_search(X,y,m)

In [36]:
print_feature_search(DT_final[-2],DT_final[-1],"{} features finales más importantes".format(m))

30 FEATURES FINALES MÁS IMPORTANTES

             feature           importance

  0.1755400086171204		       has_continent
 0.15054929426163055		   keywords_quantity
 0.11420304168699172		      continent_NA_y
 0.11160999179272135		       keywords_mean
0.060687492612023226		       has_country_y
 0.05903442880892951		           has_state
 0.03992964348976131		   keyword_frequency
 0.03973982996931458		    state_California
0.039509884721531954		     #silabas_binned
 0.03916158536280864		    #palabras_unicas
 0.02314313752702842		domain_feedsportal_x
  0.0205382176671502		              K_fire
0.016872774582141502		         tag_reunion
0.014656013207874463		      tag_earthquake
0.014435322204497033		        tag_bomber_y
0.013692794719352024		         #capitalize
 0.01091251153163281		            #silabas
0.009525449891168544		            tag_fire
0.006820329184213138		   cant_failed_links
0.005616905534291019		          #stopwords
0.004623539797052467		            tag_ebay
0.004213172290140

In [37]:
# m = 30 y min_importance = 0.00001
df_final.to_csv('features/feature_selection/features_by_random_forest.csv', encoding='utf-8')